In [41]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = "85666668775314343"

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

tm = TsetlinMachine(train_x.shape[1], 5)

out_1 = tm.forward(train_x)
tm.update_batch(train_y)
out_2 = tm.forward(train_x)
print(out_2, train_y)
assert torch.equal(train_y,out_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensor([1, 1, 1, 1, 0, 0, 0, 0]) tensor([1, 1, 1, 1, 0, 0, 0, 0])


In [42]:
tm.l1.W, tm.l2.W, tm.l3.W

(tensor([[1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [45]:
for _ in range(10000):
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    # print(seed)
    
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    
    tm = TsetlinMachine(train_x.shape[1], 5)
    out_1 = tm.forward(shuffled_x)
    tm.update_batch(shuffled_y)
    out_2 = tm.forward(shuffled_x)
    assert torch.equal(shuffled_y,out_2)